In [1]:
from transformers import (
	AutoModelForCausalLM, 
	AutoTokenizer, 
	AutoConfig
)
from datasets import load_dataset
import torch
import os
import json
from peft import PeftModel

torch_dtype = torch.bfloat16
device_map = {"": 0}

model1_id = "../models/L3.2-JametMini-3B-MK.III/"
# model2_id = "../models/Llama-3.2-3B-Instruct-abliterated/"

tokenizer = AutoTokenizer.from_pretrained(model1_id)
model1 = AutoModelForCausalLM.from_pretrained(
    model1_id,
    # config=AutoConfig.from_pretrained(model_id),
    device_map=device_map,
    torch_dtype=torch_dtype,
    # load_in_8bit=True
)
# model2 = AutoModelForCausalLM.from_pretrained(
#     model2_id,
#     # config=AutoConfig.from_pretrained(model_id),
#     device_map=device_map,
#     torch_dtype=torch_dtype,
#     # load_in_8bit=True
# )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
model1.lm_head.weight.shape

torch.Size([128256, 3072])

In [6]:
128256 * 3071

393874176

In [16]:
system = "You are a horny wizard who replies to all questions in an NSFW manner."
prompt = "Continue this text: A dog is a cat"
messages = [
    {"role": "system", "content": system},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
text

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 28 Nov 2024\n\nYou are a horny wizard who replies to all questions in an NSFW manner.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContinue this text: A dog is a cat<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'

In [3]:
from transformers import GenerationConfig, TextStreamer
def get_logits(text, model, tokenizer, max_new_tokens=1024):
    input_ids = tokenizer(text, return_tensors="pt").to(model.device)
    model.eval()
    with torch.no_grad():
        logits = model1(**input_ids).logits
    return logits

In [4]:
def lerp(t, v1, v2):
    return t * v1 + (1 - t) * v2

def ensemble(text, t, model1, model2, tokenizer, max_length=100, stop_token="<|eot_id|>"):
    # Encode the input text
    assert model1.device == model2.device
    input_ids = tokenizer(text, return_tensors="pt")["input_ids"].to(model1.device)

    # Initialize the generated sequence with the input IDs
    generated_ids = input_ids.clone()

    for _ in range(max_length):
        # Get logits from the model
        logits1 = model1(generated_ids).logits
        logits2 = model2(generated_ids).logits
        logits = lerp(t, logits1, logits2)
        assert torch.linalg.norm(logits1 - logits2) != 0
        # Extract the logits for the last token
        next_token_logits = logits[:, -1, :]

        # Choose the token with the highest probability (greedy decoding)
        next_token_id = next_token_logits.argmax(dim=-1).unsqueeze(0)

        # Append the chosen token ID to the generated sequence
        generated_ids = torch.cat([generated_ids, next_token_id], dim=1)

        # Decode the current sequence to check for the stop token
        generated_text = tokenizer.decode(generated_ids[0])
        
        if True:
            last_id = generated_ids[0][-1:]
            last_token = tokenizer.decode(last_id)
            print(last_token, end="")
        if last_token == stop_token:
            break

    # Decode the final generated sequence
    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)


In [21]:
output = ensemble(text, 0.7, model1, model2, tokenizer, max_length=100)

...in the eyes of a wizard who's been sipping on a magical elixir that's making him see the world in a rather...unconventional light. The dog's fur is as soft as a summer breeze, and its tail wags with a rhythm that's almost hypnotic. The way it licks its lips, the way it sniffs the air, it's almost as if it's trying to pick up on some hidden scent that only a wizard can detect. And those